In [1]:
import albumentations as A
import matplotlib.pyplot as plt
import torch
import cv2
from matplotlib.patches import Rectangle
import os
import PIL 
import numpy as np
# rm -rf `find -type d -name .ipynb_checkpoints`

In [ ]:
filename = 

In [2]:
classes = {c: len(os.listdir(f'archive/{c}/'))//2 for c in os.listdir('archive/')}
classes

{'rhino': 376, 'zebra': 376, 'buffalo': 376, 'elephant': 376}

In [3]:
def generate_transform(width, height): 
    h = int(height * np.random.uniform(0.5, 1))
    w = int(width * np.random.uniform(0.5, 1))

    return A.Compose([
        A.RandomCrop(width=w, height=h),
        A.RandomBrightnessContrast(p=0.2),
        A.OneOf([
                              A.HorizontalFlip(p=0.5),
                              A.MotionBlur(p=0.5),            
        ], p=1),
        A.OneOf([
                              A.OpticalDistortion(p=0.5),
                              A.GaussNoise(p=0.5)                 
        ], p=1),

    ],  bbox_params=A.BboxParams(format='yolo', min_area=1024, min_visibility=0.1, label_fields=['class_labels']))

In [13]:
for idx, c in enumerate(classes.keys()): 
    path = f'{filename}/train/'
    fnames = os.listdir(path)
    fnum = set(f.split('.')[0] for f in fnames) # number of image
    num_img = len(fnum) 
    for i, f in enumerate(fnum): 
        image_path = f"{path}{f}.jpg"
        label_path = f"{path}{f}.txt"
        print(image_path)
        # prepare images for transform
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w = image.shape[:-1]
        
        # prepare bboxes for transform
        bounding_boxes = []
        with open(label_path) as f: # open the image 
            for label in f.readlines():
                class_label, x, y, width, height = [
                    float(x) if float(x) != int(float(x)) else int(x)
                    for x in label.replace("\n", "").split()
                ]

                bounding_boxes.append([class_label, x, y, width, height])
              
        # separate bboxes from class labels for transform
        bboxes = []
        class_labels = []
        for idx in range(len(bounding_boxes)): 
            bboxes.append(bounding_boxes[idx][1:])
            class_labels.append(bounding_boxes[idx][:1])
        
        # apply transforms
        transform = generate_transform(w, h)
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']
        transformed_class_labels = transformed['class_labels']
        
        # join class and bboxes again 
        transformed_labels = []
        for idx, b in enumerate(transformed_bboxes): 
            box = list(b)
            box.insert(0,transformed_class_labels[idx][0])
            box = ''.join([str(x) + ' ' for x in box])
            transformed_labels.append(box)
            transformed_labels.append('\n')
        transformed_labels = ''.join(transformed_labels)
        
        name = str(int(i)+num_img)
        new_image = PIL.Image.fromarray(transformed_image)
        new_image.save(f'{path}{name}.jpg')
        print(f'saving: {path}{name}.jpg')
        
        with open(f'{path}{name}.txt', "w") as output:
            output.write(str(transformed_labels))

        break


archive/rhino/153.jpg
saving: archive/rhino/476.jpg
archive/zebra/153.jpg
saving: archive/zebra/376.jpg
archive/buffalo/153.jpg
saving: archive/buffalo/376.jpg
archive/elephant/153.jpg
saving: archive/elephant/376.jpg
